In [2]:
import datetime
import os
import sqlite3
import pandas as pd
import os
from platform import python_version
from datetime import datetime


import warnings
warnings.filterwarnings('ignore')

print(os.getcwd())

c:\Users\spunk\OneDrive\Janosh\ZHAW\Module\Semester 4\SCI\Challenge\SP Project\sp_project_janosh_marius\backend_sp_db


In [8]:
import os
import sqlite3

exportdir = ['Imports', 'Exports', 'Database']

notebook_dir = os.getcwd()  

for exportdirs in exportdir:
    parent_dir = os.path.dirname(notebook_dir)
    output_dir = os.path.join(parent_dir, 'backend_sp_db', exportdirs)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)



In [5]:
conn = sqlite3.connect("Database/project_SP.db")
conn.execute('''CREATE TABLE IF NOT EXISTS SP_Project
             (ID INT PRIMARY KEY     NOT NULL);''')

In [107]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("Database/project_SP.db")

# List of months
months = ["Januar", "Februar", "März", "April", "Mai", "Juni", "Juli", "August", "September", "Oktober", "November", "Dezember"]

df_sum = pd.DataFrame(columns=['Month'])

#set filter to empty string to generate for ALL Teams
filter = "DST OPS"


for month in months:
    df_cust = pd.read_excel(f"Imports/{month}_21.xlsx")

    #Can be Used to filter Abteilung to DST IPS, DST NF or DST OP
    if (filter != ""):
        df_cust = df_cust[df_cust["Abteilung (kurz)"] == filter]  

 

    # Calculate the sum of the values
    total_sum_ÜS = round(df_cust['Saldo ÜS'].sum(), 2)
    total_sum_GLAZ = round(df_cust['Saldo GLAZ'].sum(), 2)
    total_sum_Covid = round(df_cust['Saldo Covid'].sum(), 2)
    total_sum_Ist = round(df_cust['Ist'].sum(), 2)
    total_sum_Soll = round(df_cust['Soll'].sum(),2)
    total_sum_DIS = round(df_cust['Differenz Ist/Soll'].sum(),2)
    total_sum_Ferien = round(df_cust['(5010) Ferien - Saldo'].sum(),2)

    df_sum = df_sum.append({'Month': month, 'Saldo Überstunden': total_sum_ÜS, 'Saldo Glaz': total_sum_GLAZ, 'Saldo Covid': total_sum_Covid, 'Saldo IST': total_sum_Ist
    , 'Saldo SOLL': total_sum_Soll,'Saldo DIS': total_sum_DIS, 'Saldo Ferien': total_sum_Ferien }, ignore_index=True)

df_sum.to_sql(f"SP_Project_Sums_{filter}", conn, if_exists="replace", index=False)

conn.commit()
conn.close()


ADD OWN TOKEN TO ACCESS_TOKEN Parameter

In [9]:
import requests
import pandas as pd
import tempfile
import sqlite3

repo_owner = "openZH"

repo_name = "covid_19"

file_path = "fallzahlen_kanton_total_csv/COVID19_Fallzahlen_Kanton_ZH_total.csv"

access_token =  "" #ADD OWN TOKEN! 

api_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents/{file_path}"

headers = {}

if access_token:

    headers["Authorization"] = f"Bearer {access_token}"

response = requests.get(api_url, headers=headers)


if response.status_code == 200:

    data = response.json()

    download_url = data["download_url"]

    file_response = requests.get(download_url)


    if file_response.status_code == 200:

        file_content = file_response.text

    else:

        print("File download failed with status code:", file_response.status_code)

else:

    print("Request failed with status code:", response.status_code)

    print("Error message:", response.text)

# Save the file_content as a temporary file
with tempfile.NamedTemporaryFile(delete=False) as temp_file:
    temp_file.write(file_content.encode())
    temp_file_name = temp_file.name

df = pd.read_csv(temp_file_name)

selected_columns = ['date', 'ncumul_conf', 'ncumul_hosp', 'ncumul_ICU', 'ncumul_deceased']

df_selected = df[selected_columns]

df_selected['date'] = pd.to_datetime(df_selected['date']).dt.date 

targetdates = ['2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30', 
               '2021-05-31', '2021-06-30', '2021-07-31', '2021-08-31', '2021-09-30', 
               '2021-10-31', '2021-11-30', '2021-12-31']


combined_df = pd.DataFrame(columns=selected_columns)

for targetdate in targetdates:
    target_date = pd.to_datetime(targetdate)
    df_filtered = df_selected[df_selected['date'] == target_date]


    combined_df = pd.concat([combined_df, df_filtered])

print(combined_df)

df_covid = pd.DataFrame(columns=['date', 'cases', 'hospitalisation', 'ICU', 'deaths'])
df_covid['date'] = combined_df['date']
df_covid['cases'] = combined_df['ncumul_conf'].astype(int)  # Convert cases column to integers
df_covid['hospitalisation'] = combined_df['ncumul_hosp'].astype(int) 
df_covid['ICU'] = combined_df['ncumul_ICU'].astype(int)
df_covid['deaths'] = combined_df['ncumul_deceased'].astype(int)

conn = sqlite3.connect("Database/project_SP.db")

df_covid.to_sql("Covid_Data", conn, if_exists="replace", index=False)

conn.commit()
conn.close()

           date ncumul_conf  ncumul_hosp  ncumul_ICU  ncumul_deceased
309  2021-01-01       70233        445.0        96.0            900.0
339  2021-01-31       82521        229.0        58.0           1171.0
367  2021-02-28       87545        126.0        25.0           1272.0
398  2021-03-31       95338        183.0        30.0           1333.0
428  2021-04-30      106775        210.0        67.0           1377.0
459  2021-05-31      111675         93.0        39.0           1402.0
489  2021-06-30      112897         21.0        15.0           1417.0
520  2021-07-31      116327         46.0        16.0           1418.0
551  2021-08-31      127460        192.0        67.0           1436.0
581  2021-09-30      139402        106.0        38.0           1476.0
612  2021-10-31      145236         67.0        20.0           1497.0
642  2021-11-30      168497        177.0        42.0           1540.0
673  2021-12-31      218717        171.0        52.0           1619.0


In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')